In [5]:
!pip install plotly_express
!pip install pyDOE2
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.datasets import load_iris
import plotly_express as px
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from pyDOE2 import lhs
from sklearn.metrics import accuracy_score
iris = load_iris()

In [15]:
iris = load_iris()

X = iris.data
Y = iris.target

def KNN_sampling(X, y, k):
    train_sample = []
    Y = []

    n_samples = X.shape[0]

    while n_samples > k:
        neigh = KNeighborsClassifier(n_neighbors=k, algorithm='ball_tree').fit(X, y)
        indices = neigh.kneighbors(X, return_distance=False)

        # Calculate MNN and NN Scores
        neighbors = [set() for _ in range(n_samples)]
        M_neighbors = [set() for _ in range(n_samples)]
        MNN_scores = [0] * n_samples
        NN_scores = [0] * n_samples

        for i in range(n_samples):
            mutual_neighbors = set(indices[i][1:]).intersection(*[indices[j][1:] for j in indices[i]])

            for j in mutual_neighbors:
                if i != j:
                    neighbors[j].add(i)
                    M_neighbors[i].add(j)
                    MNN_scores[i] += 1

            NN_scores[i] = len(neighbors[i])

        # Calculate Index
        max_nn_score = max(NN_scores)
        index = [i for i, x in enumerate(NN_scores) if x == max_nn_score]

        if len(index) > 1:
            temp_index = [MNN_scores[i] for i in index]
            train_index = index[temp_index.index(max(temp_index))]
        else:
            train_index = index[0]

        train_sample.append(X[train_index])
        Y.append(y[train_index])

        X = np.delete(X, train_index, axis=0)
        y = np.delete(y, train_index)

        for n_index in M_neighbors[train_index]:
            if n_index < X.shape[0]:
                X = np.delete(X, n_index, axis=0)
                y = np.delete(y, n_index)

        n_samples = X.shape[0]

    # Sampling for remaining X
    if n_samples != 0:
        num_samples = X.shape[0]
        num_dimensions = X.shape[1]
        lhs_samples = lhs(num_dimensions, num_samples)
        Y.extend(y)
        train_sample.extend(X)

    return train_sample, Y

# KNN Sampling
train_sample, train_sample_result = KNN_sampling(X, Y, 10)

# TSNE for HD to LD
tsne_results = []
test_tsne_results = []

train_sample_tsne = TSNE(n_components=3, perplexity=10, random_state=32).fit_transform(np.array(train_sample))
test_sample_tsne = TSNE(n_components=3, perplexity=10, random_state=32).fit_transform(np.array(iris.data))

tsne_results.append(train_sample_tsne)
test_tsne_results.append(test_sample_tsne)


# KNN accuracy
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(np.array(tsne_results[0]), train_sample_result)
y_pred = knn.predict(test_tsne_results[0])
accuracy = accuracy_score(Y, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.82


In [16]:
df = pd.DataFrame(iris.data,columns=iris.feature_names)
m=px.scatter_3d(df, x="sepal length (cm)", y="petal width (cm)", z="sepal width (cm)", size="petal length (cm)",
              color=iris.target, color_discrete_map = {"Joly": "blue", "Bergeron": "violet", "Coderre":"pink"})
m.show()

In [18]:
df = pd.DataFrame(np.array(tsne_results[0]),columns=["1","2","3"])
m=px.scatter_3d(df, x="1", y="2", z="3",
              color=train_sample_result, color_discrete_map = {"Joly": "blue", "Bergeron": "violet", "Coderre":"pink"})
m.show()